Выберем необходимые нам строки из прошлого дз для реализации регресии

In [1]:
from sklearn.datasets import load_boston
import pandas as pd
import numpy as np
data = pd.read_csv(load_boston()['filename'], skiprows=1)

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/feature_extraction/image.py:167: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  dtype=np.int):


In [2]:
data.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [3]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:30: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method='lar', copy_X=True, eps=np.finfo(np.float).eps,
/srv/conda/envs/notebook/lib/python3.7/site-packages/sklearn/linear_model/least_angle.py:167: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  method

In [4]:
def get_score(X,y, random_seed=42, model=None):
    '''Функция возвращает % качества работы модели'''
    if model is None:
        model = LinearRegression()
        X_train, X_test, y_train, y_test=train_test_split(X, y, test_size=0.3, random_state=random_seed)
        model.fit(X_train, y_train)
        return model.score(X_test, y_test)

In [5]:
columns = ['CHAS',  'RM', 'RAD', 'TAX', 'LSTAT', 'PTRATIO', 'NOX',  'DIS',  'ZN']

#### наш результат регресии:

In [6]:
get_score(data[columns], data['MEDV'])

0.7183142507178601

In [7]:
data_reg = data.copy()
data_reg.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,MEDV
0,0.00632,18.0,2.31,0,0.538,6.575,65.2,4.0900,1,296,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0,0.469,6.421,78.9,4.9671,2,242,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0,0.469,7.185,61.1,4.9671,2,242,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0,0.458,6.998,45.8,6.0622,3,222,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0,0.458,7.147,54.2,6.0622,3,222,18.7,396.90,5.33,36.2


In [8]:
data_reg['taxrm'] = data_reg['TAX'] * data_reg['RM']
data_reg['taxdis'] = data_reg['TAX'] * data_reg['LSTAT']
columns_reg = ['CHAS',  'RM', 'RAD', 'TAX', 'LSTAT', 'PTRATIO', 'NOX',  'DIS',  'ZN', 'taxdis', 'taxrm']

#### наш улучшенный результат регресии:

In [9]:
get_score(data_reg[columns_reg], data_reg['MEDV'])

0.7844089601546259

Деревье решений

In [10]:
# импортирем функцию для кол-ва вхождений, чтобы использовать наши функции
from collections import Counter

$H(R) = -\sum_{k=1}^{K}p_klogp_k$

In [11]:
def HEntropy(x):
    """функция для расчёта энтропийного критерия качества"""
    length = len(x)
    cnt = Counter(x)
    
    ent = 0
    for cl in cnt.values():
        p = cl / length
        l2 = np.log2(p)
        it = -p * l2
        ent += it
    
    return ent

$Н(R) = -\sum_{k=1}^{K}p_k(1-p_k)$

In [12]:
def HEntropy(l):
    """функция расчёта энтропийного критерия множества"""
    length = len(l)
    cnt = Counter(l)
    
    ent = 0
    for cl in cnt.values():
        p = cl / length
        l2 = np.log2(p)
        it = -p * l2
        ent += it
    
    return ent

$Н(R) = -\sum_{k=1}^{K}p_k(1-p_k)$

In [13]:
def HGini(l):
    """Функция для рассчета критерия Джини"""
    length = len(l)
    cnt = Counter(l)
    
    gini = 0
    for cl in cnt.values():
        p_1 = cl / length
        p_2 = (1 - p_1)
        it = p_1 * p_2
        gini += it
    
    return gini

$IG(R) = H(R) - \frac{|R_l|}{|R|}H(R_l) - \frac{|R_r|}{|R|}H(R_r)$

In [14]:
def IG(H, l, i):
    """Information Gain (IG)* - функционал качества, отвечающий на вопрос, а сколько энтропии мы погасили при определённом разбиении? 
    На каждом шаге разбиения при построении дерева максимизируется IG"""
    left_l = l[:i]
    right_l = l[i:]
    return H(l) - (len(left_l) / len(l)) * H(left_l) - (len(right_l) / len(l)) * H(right_l)


In [15]:
def test_H(H, l):
    """функция для визуализации работы произвольного критерия качества на выборке"""
    print("{:5} {:3}   {:4} {:4} {:4}".format("#","l","IG","Hl","Hr"))
    print("-"*24)
    i_max,IG_max=0,0
    for i in range(0,len(l)):
        print(f"{i:2}. {l[i]:3}   {IG(H, l, i):.2f} {H(l[:i]):.2f} {H(l[i:]):.2f}")
        if IG_max < IG(H, l, i):
            i_max, IG_max = i, IG(H, l, i)
    print(f'Деление после элемента:{i_max-1}')

In [16]:
# запишем нашу целевую переменную в массив
medv = np.array(data.MEDV)
# отсортируем переменную
sort_medv = sorted(medv)

In [17]:
HEntropy(medv)

7.527082109018076

In [18]:
# посмотрим энтропию
test_H(HEntropy, medv)

#     l     IG   Hl   Hr  
------------------------
 0. 24.0   0.00 0.00 7.53
 1. 21.6   0.02 0.00 7.53
 2. 34.7   0.03 1.00 7.52
 3. 33.4   0.04 1.58 7.52
 4. 36.2   0.05 2.00 7.52
 5. 28.7   0.06 2.32 7.51
 6. 22.9   0.07 2.58 7.51
 7. 27.1   0.08 2.81 7.51
 8. 16.5   0.09 3.00 7.51
 9. 18.9   0.09 3.17 7.51
10. 15.0   0.10 3.32 7.51
11. 18.9   0.10 3.46 7.51
12. 21.7   0.11 3.42 7.51
13. 20.4   0.12 3.55 7.51
14. 18.2   0.12 3.66 7.51
15. 19.9   0.12 3.77 7.51
16. 23.1   0.13 3.88 7.52
17. 17.5   0.13 3.97 7.52
18. 20.2   0.13 4.06 7.52
19. 18.2   0.14 4.14 7.52
20. 13.6   0.15 4.12 7.51
21. 19.6   0.15 4.20 7.51
22. 15.2   0.15 4.28 7.51
23. 14.5   0.16 4.35 7.51
24. 15.6   0.16 4.42 7.51
25. 13.9   0.16 4.48 7.52
26. 16.6   0.17 4.55 7.51
27. 14.8   0.17 4.61 7.51
28. 18.4   0.18 4.66 7.51
29. 21.0   0.18 4.72 7.51
30. 12.7   0.18 4.77 7.51
31. 14.5   0.19 4.83 7.51
32. 13.2   0.19 4.81 7.50
33. 13.1   0.20 4.86 7.50
34. 13.5   0.20 4.91 7.50
35. 18.9   0.21 4.96 7.50
36. 20.0   0

In [19]:
# посмотрим на информационный выигрыш
IG(HEntropy, medv, 372)

0.53518054886991

In [20]:
HGini(medv)

0.9931181552594163

In [21]:
# посмотрим на критерий Джини
test_H(HGini, medv)

#     l     IG   Hl   Hr  
------------------------
 0. 24.0   0.00 0.00 0.99
 1. 21.6   0.00 0.00 0.99
 2. 34.7   0.00 0.50 0.99
 3. 33.4   0.00 0.67 0.99
 4. 36.2   0.00 0.75 0.99
 5. 28.7   0.00 0.80 0.99
 6. 22.9   0.00 0.83 0.99
 7. 27.1   0.00 0.86 0.99
 8. 16.5   0.00 0.88 0.99
 9. 18.9   0.00 0.89 0.99
10. 15.0   0.00 0.90 0.99
11. 18.9   0.00 0.91 0.99
12. 21.7   0.00 0.90 0.99
13. 20.4   0.00 0.91 0.99
14. 18.2   0.00 0.92 0.99
15. 19.9   0.00 0.92 0.99
16. 23.1   0.00 0.93 0.99
17. 17.5   0.00 0.93 0.99
18. 20.2   0.00 0.94 0.99
19. 18.2   0.00 0.94 0.99
20. 13.6   0.00 0.94 0.99
21. 19.6   0.00 0.94 0.99
22. 15.2   0.00 0.95 0.99
23. 14.5   0.00 0.95 0.99
24. 15.6   0.00 0.95 0.99
25. 13.9   0.00 0.95 0.99
26. 16.6   0.00 0.96 0.99
27. 14.8   0.00 0.96 0.99
28. 18.4   0.00 0.96 0.99
29. 21.0   0.00 0.96 0.99
30. 12.7   0.00 0.96 0.99
31. 14.5   0.00 0.96 0.99
32. 13.2   0.00 0.96 0.99
33. 13.1   0.00 0.96 0.99
34. 13.5   0.00 0.97 0.99
35. 18.9   0.00 0.97 0.99
36. 20.0   0

In [22]:
# посмотрим на информационный выигрыш
IG(HGini, medv, 372)

0.0031308864523600377